<a href="https://colab.research.google.com/github/FarhodjonJaborov/movie_recommendation/blob/main/Datahack_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=10113, stop_words = 'english')
from sklearn.metrics.pairwise import cosine_similarity
import random

In [ ]:
path = '/content/drive/MyDrive/DataHack/users.csv'
users = pd.read_csv(path)
users.head()

,user_id,age,income,sex,kids_flg
0,973171,age_25_34,income_60_90,М,1
1,962099,age_18_24,income_20_40,М,0
2,1047345,age_45_54,income_40_60,Ж,0
3,721985,age_45_54,income_20_40,Ж,0
4,704055,age_35_44,income_60_90,Ж,0


In [ ]:
items=pd.read_csv('/content/drive/MyDrive/Копия items.csv')
items.head(3)

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ..."
1,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео..."
2,10716,film,Тактическая сила,Tactical Force,2011.0,"криминал, зарубежные, триллеры, боевики, комедии",Канада,NaN,16.0,NaN,Адам П. Калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг..."


In [ ]:
interactions = pd.read_csv('/content/drive/MyDrive/Копия interactions.csv')
interactions.head()

,user_id,item_id,last_watch_dt,total_dur,watched_pct
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0
3,864613,7638,2021-07-05,14483,100.0
4,964868,9506,2021-04-30,6725,100.0


In [ ]:
movies = pd.merge(interactions, items, on='item_id')
movies = pd.merge(movies, users, on='user_id')
movies.head(3)

,user_id,item_id,last_watch_dt,total_dur,watched_pct,content_type,title,title_orig,release_year,genres,...,age_rating,studios,directors,actors,description,keywords,age,income,sex,kids_flg
0,176549,9506,2021-05-11,4250,72.0,film,Холодное сердце,Frozen,2013.0,"фэнтези, мультфильм, музыкальные",...,0.0,NaN,"Крис Бак, Дженнифер Ли","Кристен Белл, Идина Мензел, Джонатан Грофф, Дж...","Когда сбывается древнее предсказание, и короле...","королева, мюзикл, принцесса, предательство, сн...",age_35_44,income_40_60,М,0
1,176549,1659,2021-05-16,5147,100.0,film,Три богатыря. Ход конем,Tri bogatyrya. Khod konem,2014.0,"мультфильм, фэнтези, приключения, комедии",...,6.0,NaN,К. Феоктистов,"Сергей Маковецкий, Дмитрий Высоцкий, Дмитрий Н...",Придворный конь Гай Юлий Цезарь на свою беду п...,"2014, россия, три, богатыря, ход, конем",age_35_44,income_40_60,М,0
2,176549,8437,2021-06-21,773,11.0,film,Капитан Марвел,Captain Marvel,2019.0,"боевики, фантастика",...,16.0,NaN,"Анна Боден, Райан Флек","Бри Ларсон, Сэмюэл Л. Джексон, Бен Мендельсон,...",После столкновения с враждующими инопланетными...,"супергерой, основанный на комиксе, супергероин...",age_35_44,income_40_60,М,0


In [ ]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4424477 entries, 0 to 4424476
Data columns (total 22 columns):
 #   Column         Dtype  
---  ------         -----  
 0   user_id        int64  
 1   item_id        int64  
 2   last_watch_dt  object 
 3   total_dur      int64  
 4   watched_pct    float64
 5   content_type   object 
 6   title          object 
 7   title_orig     object 
 8   release_year   float64
 9   genres         object 
 10  countries      object 
 11  for_kids       float64
 12  age_rating     float64
 13  studios        object 
 14  directors      object 
 15  actors         object 
 16  description    object 
 17  keywords       object 
 18  age            object 
 19  income         object 
 20  sex            object 
 21  kids_flg       int64  
dtypes: float64(4), int64(4), object(14)
memory usage: 742.6+ MB


In [ ]:
movies = movies[['user_id','content_type', 'title',	'title_orig' , 'genres','directors','actors','description','keywords']]
movies.head()

,user_id,content_type,title,title_orig,genres,directors,actors,description,keywords
0,176549,film,Холодное сердце,Frozen,"фэнтези, мультфильм, музыкальные","Крис Бак, Дженнифер Ли","Кристен Белл, Идина Мензел, Джонатан Грофф, Дж...","Когда сбывается древнее предсказание, и короле...","королева, мюзикл, принцесса, предательство, сн..."
1,176549,film,Три богатыря. Ход конем,Tri bogatyrya. Khod konem,"мультфильм, фэнтези, приключения, комедии",К. Феоктистов,"Сергей Маковецкий, Дмитрий Высоцкий, Дмитрий Н...",Придворный конь Гай Юлий Цезарь на свою беду п...,"2014, россия, три, богатыря, ход, конем"
2,176549,film,Капитан Марвел,Captain Marvel,"боевики, фантастика","Анна Боден, Райан Флек","Бри Ларсон, Сэмюэл Л. Джексон, Бен Мендельсон,...",После столкновения с враждующими инопланетными...,"супергерой, основанный на комиксе, супергероин..."
3,176549,film,Гнев человеческий,Wrath of Man,"боевики, триллеры",Гай Ричи,"Джейсон Стэйтем, Холт МакКэллани, Джеффри Доно...",Грузовики лос-анджелесской инкассаторской комп...,"ограбление, криминальный авторитет, месть, пер..."
4,176549,film,Мстители: Война бесконечности,AVENGERS: INFINITY WAR,"боевики, фантастика, приключения","Энтони Руссо, Джо Руссо","Роберт Дауни мл., Крис Хемсворт, Марк Руффало,...",Пока Мстители и их союзники продолжают защищат...,"магия, жертва, супергерой, основанный на комик..."


In [ ]:
movies.isnull().sum()

user_id         0
content_type    0
title           0
title_orig      0
genres          0
directors       0
actors          0
description     0
keywords        0
dtype: int64

In [ ]:
movies.dropna(inplace=True)
movies = movies[:10113]
movies.shape

(10113, 9)

In [ ]:
movies['tags'] = movies['genres']+ movies['directors']+ movies['actors'] + movies['description'] + movies['keywords']
movies.head()

,user_id,content_type,title,title_orig,genres,directors,actors,description,keywords,tags
0,176549,film,Холодное сердце,Frozen,"фэнтези, мультфильм, музыкальные","Крис Бак, Дженнифер Ли","Кристен Белл, Идина Мензел, Джонатан Грофф, Дж...","Когда сбывается древнее предсказание, и короле...","королева, мюзикл, принцесса, предательство, сн...","фэнтези, мультфильм, музыкальныеКрис Бак, Джен..."
1,176549,film,Три богатыря. Ход конем,Tri bogatyrya. Khod konem,"мультфильм, фэнтези, приключения, комедии",К. Феоктистов,"Сергей Маковецкий, Дмитрий Высоцкий, Дмитрий Н...",Придворный конь Гай Юлий Цезарь на свою беду п...,"2014, россия, три, богатыря, ход, конем","мультфильм, фэнтези, приключения, комедииК. Фе..."
2,176549,film,Капитан Марвел,Captain Marvel,"боевики, фантастика","Анна Боден, Райан Флек","Бри Ларсон, Сэмюэл Л. Джексон, Бен Мендельсон,...",После столкновения с враждующими инопланетными...,"супергерой, основанный на комиксе, супергероин...","боевики, фантастикаАнна Боден, Райан ФлекБри Л..."
3,176549,film,Гнев человеческий,Wrath of Man,"боевики, триллеры",Гай Ричи,"Джейсон Стэйтем, Холт МакКэллани, Джеффри Доно...",Грузовики лос-анджелесской инкассаторской комп...,"ограбление, криминальный авторитет, месть, пер...","боевики, триллерыГай РичиДжейсон Стэйтем, Холт..."
4,176549,film,Мстители: Война бесконечности,AVENGERS: INFINITY WAR,"боевики, фантастика, приключения","Энтони Руссо, Джо Руссо","Роберт Дауни мл., Крис Хемсворт, Марк Руффало,...",Пока Мстители и их союзники продолжают защищат...,"магия, жертва, супергерой, основанный на комик...","боевики, фантастика, приключенияЭнтони Руссо, ..."


In [ ]:
def stem(text):
  y= []

  for i in text.split():
    y.append(ps.stem(i))

    return " ".join(y)

In [ ]:
new_df = movies[['user_id', 'title', 'tags']]
new_df.head()

,user_id,title,tags
0,176549,Холодное сердце,"фэнтези, мультфильм, музыкальныеКрис Бак, Джен..."
1,176549,Три богатыря. Ход конем,"мультфильм, фэнтези, приключения, комедииК. Фе..."
2,176549,Капитан Марвел,"боевики, фантастикаАнна Боден, Райан ФлекБри Л..."
3,176549,Гнев человеческий,"боевики, триллерыГай РичиДжейсон Стэйтем, Холт..."
4,176549,Мстители: Война бесконечности,"боевики, фантастика, приключенияЭнтони Руссо, ..."


In [ ]:
new_df['tags'] = new_df['tags'].apply(stem)

<ipython-input-14-be18a4346d89>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(stem)


In [ ]:
new_df['tags'][0]

'фэнтези,'

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

<ipython-input-16-8b60b591a07f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())


In [ ]:
new_df.head()

,user_id,title,tags
0,176549,Холодное сердце,"фэнтези,"
1,176549,Три богатыря. Ход конем,"мультфильм,"
2,176549,Капитан Марвел,"боевики,"
3,176549,Гнев человеческий,"боевики,"
4,176549,Мстители: Война бесконечности,"боевики,"


In [ ]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [ ]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
cv.get_feature_names_out()

array(['аниме', 'биография', 'боевики', 'боевикиантон', 'боевикигарет',
       'боевикиджесси', 'боевикидон', 'боевикийорк', 'боевикикааре',
       'боевикимаксим', 'боевикимартин', 'боевикиникита', 'боевикиолег',
       'боевикиричард', 'боевикиросс', 'боевикисергей', 'боевикифредерик',
       'вестерн', 'военные', 'детективы', 'детективыстанислав', 'детские',
       'детскиеандрей', 'дж', 'для', 'документальноеалексей',
       'документальноедоун', 'документальноедэймон', 'документальноелука',
       'документальноепатрис', 'документальноеричард',
       'документальноероберт', 'документальноесветлана',
       'документальноестивен', 'документальноетом', 'документальноетони',
       'документальноетьерри', 'драмы', 'драмыавдотья', 'драмыалексей',
       'драмыандрей', 'драмыанна', 'драмыантон', 'драмыарсен',
       'драмыбарри', 'драмыбартлетт', 'драмывадим', 'драмывера',
       'драмывигго', 'драмывим', 'драмывладимир', 'драмыгабриэле',
       'драмыгай', 'драмыглендин', 'драмыдавид

In [ ]:
ps.stem('фэнтези, мультфильм, музыкальныекрис бак, дженнифер ликристен белл, идина мензел, джонатан грофф, джош гад, сантино фонтана, алан тьюдик, киран хайндс, крис уильямс, стивен дж. андерсон, майа уилсон, киаран хиндс, морис ламарш, дженнифер ли, дара макгарри, фред татаскьоркогда сбывается древнее предсказание, и\xa0королевство погружается в\xa0объятия вечной зимы, трое бесстрашных героев\xa0—\xa0принцесса анна, отважный кристофф и\xa0его верный олень свен\xa0—\xa0отправляются в\xa0горы, чтобы найти сестру анны, эльзу, которая может снять со\xa0страны леденящее заклятие. по\xa0пути их\xa0ждет множество увлекательных сюрпризов и\xa0захватывающих приключений: встреча с\xa0мистическими троллями, знакомство с\xa0очаровательным снеговиком по\xa0имени олаф, горные вершины покруче эвереста и\xa0магия в\xa0каждой снежинке. анне и\xa0кристоффу предстоит сплотиться и\xa0противостоять могучей стихии, чтобы спасти королевство и\xa0тех, кто\xa0им дорог.королева, мюзикл, принцесса, предательство, снегов')

'фэнтези, мультфильм, музыкальныекрис бак, дженнифер ликристен белл, идина мензел, джонатан грофф, джош гад, сантино фонтана, алан тьюдик, киран хайндс, крис уильямс, стивен дж. андерсон, майа уилсон, киаран хиндс, морис ламарш, дженнифер ли, дара макгарри, фред татаскьоркогда сбывается древнее предсказание, и\xa0королевство погружается в\xa0объятия вечной зимы, трое бесстрашных героев\xa0—\xa0принцесса анна, отважный кристофф и\xa0его верный олень свен\xa0—\xa0отправляются в\xa0горы, чтобы найти сестру анны, эльзу, которая может снять со\xa0страны леденящее заклятие. по\xa0пути их\xa0ждет множество увлекательных сюрпризов и\xa0захватывающих приключений: встреча с\xa0мистическими троллями, знакомство с\xa0очаровательным снеговиком по\xa0имени олаф, горные вершины покруче эвереста и\xa0магия в\xa0каждой снежинке. анне и\xa0кристоффу предстоит сплотиться и\xa0противостоять могучей стихии, чтобы спасти королевство и\xa0тех, кто\xa0им дорог.королева, мюзикл, принцесса, предательство, снего

In [ ]:
similarity = cosine_similarity(vectors)

In [ ]:
sorted(list(enumerate(similarity[0])), reverse = True, key=lambda x:x[1])[1:6]

[(54, 1.0), (81, 1.0), (84, 1.0), (93, 1.0), (97, 1.0)]

In [ ]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:11]

    recommended_indices = [i[0] for i in movies_list]
    recommended_indices = [idx for idx in recommended_indices if idx != movie_index]
    random_movies = random.sample(recommended_indices, min(10, len(recommended_indices)))
    recommended_movies = [new_df.iloc[idx].title for idx in random_movies]

    for movie in recommended_movies:
        print(movie)

Возьмите название фильма из title и поместите его посередине "". И программа порекомендует вам фильмы

In [ ]:
recommend('Капитан Марвел')

Джуманджи: Зов джунглей
Великий уравнитель 2
Гнев человеческий
Курьер
Быстрее пули
Пистолера
Мстители: Война бесконечности
Час истины
Балканский рубеж
Дочь волка


In [ ]:
new_df

,user_id,title,tags
0,176549,Холодное сердце,"фэнтези,"
1,176549,Три богатыря. Ход конем,"мультфильм,"
2,176549,Капитан Марвел,"боевики,"
3,176549,Гнев человеческий,"боевики,"
4,176549,Мстители: Война бесконечности,"боевики,"
...,...,...,...
11213,1034277,Тачки 3,"мультфильм,"
11214,1034277,ВАЛЛ-И,"фантастика,"
11217,1034277,Падал прошлогодний снег,"драмы,"
11218,1034277,Алеша Попович и Тугарин Змей,"мультфильм,"
